# MovieMind - Exploratory Data Analysis

This notebook performs comprehensive EDA on movie reviews and metadata.

## Contents:
1. Data Loading
2. Univariate Analysis
3. Bivariate Analysis
4. Genre Analysis
5. Temporal Analysis
6. Geographic Analysis (optional)
7. Correlation Analysis

In [ ]:
# Imports
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from src.utils.db_manager import DatabaseManager

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Data Loading

In [ ]:
# Load data from PostgreSQL
with DatabaseManager() as db:
    # Load movies
    movies_query = "SELECT * FROM movies LIMIT 1000"
    df_movies = pd.DataFrame(db.execute_query(movies_query))
    
    # Load reviews
    reviews_query = "SELECT * FROM reviews LIMIT 5000"
    df_reviews = pd.DataFrame(db.execute_query(reviews_query))
    
    # Load aggregated view
    stats_query = "SELECT * FROM movie_review_stats LIMIT 1000"
    df_stats = pd.DataFrame(db.execute_query(stats_query))

print(f"Movies loaded: {len(df_movies)}")
print(f"Reviews loaded: {len(df_reviews)}")
print(f"Stats loaded: {len(df_stats)}")

In [ ]:
# Display sample data
print("\n=== Movies Sample ===")
display(df_movies.head())

print("\n=== Reviews Sample ===")
display(df_reviews.head())

print("\n=== Stats Sample ===")
display(df_stats.head())

In [ ]:
# Data info
print("\n=== Movies Info ===")
df_movies.info()

print("\n=== Reviews Info ===")
df_reviews.info()

## 2. Univariate Analysis

In [ ]:
# Descriptive statistics for movies
print("\n=== Movie Descriptive Statistics ===")
display(df_movies[['vote_average', 'vote_count', 'popularity', 'runtime', 'budget', 'revenue']].describe())

In [ ]:
# Distribution of ratings
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df_movies['vote_average'].dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Movie Ratings (TMDb)')
axes[0].axvline(df_movies['vote_average'].mean(), color='red', linestyle='--', label='Mean')
axes[0].axvline(df_movies['vote_average'].median(), color='green', linestyle='--', label='Median')
axes[0].legend()

# Box plot
axes[1].boxplot(df_movies['vote_average'].dropna())
axes[1].set_ylabel('Rating')
axes[1].set_title('Box Plot of Movie Ratings')

plt.tight_layout()
plt.show()

# Statistics
print(f"Mean rating: {df_movies['vote_average'].mean():.2f}")
print(f"Median rating: {df_movies['vote_average'].median():.2f}")
print(f"Std deviation: {df_movies['vote_average'].std():.2f}")

In [ ]:
# Runtime distribution
plt.figure(figsize=(12, 5))

runtime_data = df_movies['runtime'].dropna()
runtime_data = runtime_data[runtime_data > 0]  # Remove invalid values

plt.hist(runtime_data, bins=40, edgecolor='black', alpha=0.7)
plt.xlabel('Runtime (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of Movie Runtime')
plt.axvline(runtime_data.mean(), color='red', linestyle='--', label=f'Mean: {runtime_data.mean():.0f} min')
plt.axvline(runtime_data.median(), color='green', linestyle='--', label=f'Median: {runtime_data.median():.0f} min')
plt.legend()
plt.show()

In [ ]:
# Review text length distribution
if 'text_length' in df_reviews.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(df_reviews['text_length'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Review Length (characters)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Review Length')
    
    # Log scale
    axes[1].hist(df_reviews['text_length'], bins=50, edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Review Length (characters)')
    axes[1].set_ylabel('Frequency (log scale)')
    axes[1].set_title('Distribution of Review Length (Log Scale)')
    axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Average review length: {df_reviews['text_length'].mean():.0f} characters")
    print(f"Median review length: {df_reviews['text_length'].median():.0f} characters")

## 3. Bivariate Analysis

In [ ]:
# Correlation heatmap for numerical features
numeric_cols = ['vote_average', 'vote_count', 'popularity', 'runtime', 'budget', 'revenue']
corr_data = df_movies[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_data, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1)
plt.title('Correlation Heatmap of Movie Features')
plt.tight_layout()
plt.show()

In [ ]:
# Runtime vs Rating
plt.figure(figsize=(12, 6))
plt.scatter(df_movies['runtime'], df_movies['vote_average'], alpha=0.3)
plt.xlabel('Runtime (minutes)')
plt.ylabel('Rating')
plt.title('Runtime vs Rating')

# Add trend line
valid_data = df_movies[['runtime', 'vote_average']].dropna()
z = np.polyfit(valid_data['runtime'], valid_data['vote_average'], 1)
p = np.poly1d(z)
plt.plot(valid_data['runtime'].sort_values(), p(valid_data['runtime'].sort_values()), 
         "r--", alpha=0.8, label='Trend line')
plt.legend()
plt.show()

# Correlation
corr = valid_data['runtime'].corr(valid_data['vote_average'])
print(f"Correlation between runtime and rating: {corr:.3f}")

In [ ]:
# Budget vs Revenue (if available)
budget_revenue = df_movies[['budget', 'revenue']].dropna()
budget_revenue = budget_revenue[(budget_revenue['budget'] > 0) & (budget_revenue['revenue'] > 0)]

if len(budget_revenue) > 0:
    plt.figure(figsize=(12, 6))
    plt.scatter(budget_revenue['budget'], budget_revenue['revenue'], alpha=0.5)
    plt.xlabel('Budget ($)')
    plt.ylabel('Revenue ($)')
    plt.title('Budget vs Revenue')
    plt.xscale('log')
    plt.yscale('log')
    
    # ROI line (break-even)
    max_val = max(budget_revenue['budget'].max(), budget_revenue['revenue'].max())
    plt.plot([1, max_val], [1, max_val], 'r--', alpha=0.5, label='Break-even line')
    plt.legend()
    plt.show()
    
    # ROI calculation
    budget_revenue['roi'] = (budget_revenue['revenue'] - budget_revenue['budget']) / budget_revenue['budget'] * 100
    print(f"Average ROI: {budget_revenue['roi'].mean():.1f}%")
    print(f"Median ROI: {budget_revenue['roi'].median():.1f}%")

## 4. Genre Analysis

In [ ]:
# Extract and count genres
from collections import Counter

all_genres = []
for genres in df_movies['genres'].dropna():
    if isinstance(genres, list):
        all_genres.extend(genres)

genre_counts = Counter(all_genres)
genre_df = pd.DataFrame(genre_counts.most_common(15), columns=['Genre', 'Count'])

# Plot genre distribution
plt.figure(figsize=(12, 6))
plt.barh(genre_df['Genre'], genre_df['Count'])
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.title('Top 15 Most Common Genres')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Average rating by genre
genre_ratings = {}

for idx, row in df_movies.iterrows():
    if pd.notna(row['genres']) and pd.notna(row['vote_average']):
        if isinstance(row['genres'], list):
            for genre in row['genres']:
                if genre not in genre_ratings:
                    genre_ratings[genre] = []
                genre_ratings[genre].append(row['vote_average'])

# Calculate average ratings
genre_avg = {genre: np.mean(ratings) for genre, ratings in genre_ratings.items() if len(ratings) > 10}
genre_avg_df = pd.DataFrame(list(genre_avg.items()), columns=['Genre', 'Avg_Rating'])
genre_avg_df = genre_avg_df.sort_values('Avg_Rating', ascending=False)

# Plot
plt.figure(figsize=(12, 6))
plt.barh(genre_avg_df['Genre'], genre_avg_df['Avg_Rating'])
plt.xlabel('Average Rating')
plt.ylabel('Genre')
plt.title('Average Rating by Genre')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. Temporal Analysis

In [ ]:
# Convert release_date to datetime
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')
df_movies['release_year'] = df_movies['release_date'].dt.year

# Movies per year
movies_per_year = df_movies.groupby('release_year').size()

plt.figure(figsize=(14, 6))
movies_per_year.plot(kind='line', marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Movies')
plt.title('Number of Movies Released Per Year')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Average rating over time
avg_rating_per_year = df_movies.groupby('release_year')['vote_average'].mean()

plt.figure(figsize=(14, 6))
avg_rating_per_year.plot(kind='line', marker='o', color='green')
plt.xlabel('Year')
plt.ylabel('Average Rating')
plt.title('Average Movie Rating Over Time')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Statistical Tests

In [ ]:
# Chi-squared test: Genre vs High/Low Rating
# Create binary rating (high/low)
df_movies['rating_category'] = df_movies['vote_average'].apply(
    lambda x: 'High' if x >= 7.0 else 'Low' if pd.notna(x) else None
)

# Create genre-rating contingency table for top genres
top_genres = ['Drama', 'Comedy', 'Action', 'Thriller', 'Romance']

for genre in top_genres:
    df_movies[f'is_{genre}'] = df_movies['genres'].apply(
        lambda x: genre in x if isinstance(x, list) else False
    )

print("\n=== Chi-Squared Tests (Genre vs Rating Category) ===")
for genre in top_genres:
    contingency = pd.crosstab(
        df_movies[f'is_{genre}'],
        df_movies['rating_category']
    )
    
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
    print(f"\n{genre}:")
    print(f"  Chi² = {chi2:.4f}")
    print(f"  p-value = {p_value:.4f}")
    print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")

In [ ]:
# ANOVA: Rating differences across genres
# Prepare data for top 5 genres
genre_groups = []
for genre in top_genres:
    genre_ratings_list = df_movies[df_movies[f'is_{genre}']]['vote_average'].dropna()
    genre_groups.append(genre_ratings_list)

# Perform ANOVA
f_stat, p_value = stats.f_oneway(*genre_groups)

print("\n=== ANOVA Test (Rating across genres) ===")
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

# Box plot for visualization
plt.figure(figsize=(12, 6))
data_for_plot = [group for group in genre_groups]
plt.boxplot(data_for_plot, labels=top_genres)
plt.ylabel('Rating')
plt.xlabel('Genre')
plt.title('Rating Distribution by Genre')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation tests with p-values
print("\n=== Correlation Tests ===")

# Runtime vs Rating
valid_data = df_movies[['runtime', 'vote_average']].dropna()
corr, p_value = stats.pearsonr(valid_data['runtime'], valid_data['vote_average'])
print(f"\nRuntime vs Rating:")
print(f"  Correlation: {corr:.4f}")
print(f"  p-value: {p_value:.4f}")
print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")

# Vote count vs Rating
valid_data2 = df_movies[['vote_count', 'vote_average']].dropna()
corr2, p_value2 = stats.pearsonr(valid_data2['vote_count'], valid_data2['vote_average'])
print(f"\nVote Count vs Rating:")
print(f"  Correlation: {corr2:.4f}")
print(f"  p-value: {p_value2:.4f}")
print(f"  Significant: {'Yes' if p_value2 < 0.05 else 'No'}")

## 7. Summary Statistics

In [ ]:
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

print(f"\nTotal movies: {len(df_movies)}")
print(f"Total reviews: {len(df_reviews)}")

print(f"\nRating statistics:")
print(f"  Mean: {df_movies['vote_average'].mean():.2f}")
print(f"  Median: {df_movies['vote_average'].median():.2f}")
print(f"  Std Dev: {df_movies['vote_average'].std():.2f}")
print(f"  Min: {df_movies['vote_average'].min():.2f}")
print(f"  Max: {df_movies['vote_average'].max():.2f}")

print(f"\nMost common genre: {genre_df.iloc[0]['Genre']}")
print(f"Highest rated genre: {genre_avg_df.iloc[0]['Genre']} ({genre_avg_df.iloc[0]['Avg_Rating']:.2f})")

if 'text_length' in df_reviews.columns:
    print(f"\nAverage review length: {df_reviews['text_length'].mean():.0f} characters")
    print(f"Median review length: {df_reviews['text_length'].median():.0f} characters")